In [1]:
import keras
import tensorflow as tf
from tensorflow.python.ops import math_ops
import numpy as np
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import h5py
import time
import matplotlib.pyplot as plt
from keras.constraints import max_norm
from scipy.linalg import orth

Using TensorFlow backend.


In [2]:
n = 2 #This is the size of the matrix and consequently the number of neurons in each layer in the NN created
orthonormal_vectors = 10*np.random.rand(n,n)
orthonormal_vector = orth(orthonormal_vectors)
print(orthonormal_vectors)

[[8.82963324 7.16031211]
 [8.01308998 6.73334567]]


In [29]:
A = np.array([[2,5],[3,7]])
x_train = np.array([[-7,3],[5,-2]])
x_validate = np.array([[4,7]])

In [30]:
#Get test data
y_train = np.zeros([len(x_train),2])
y_validate = np.zeros([len(x_validate),2])
for i in range(0,len(x_train)):
    y_train[i] = np.matmul(A,x_train[i])
for i in range(0,len(x_validate)):
    y_validate[i] = np.matmul(A,x_validate[i])


In [31]:
def my_init(shape, dtype=None):
    return K.set_value()

In [32]:
print(x_train)

[[-7  3]
 [ 5 -2]]


In [33]:
np.shape(y_validate)

(1, 2)

In [34]:
def RegularizationLoss(Lambda):
    def loss(y_true,y_pred) :
        return(K.mean(K.square(y_pred - y_true) + Lambda*(tf.linalg.det(model.get_weights()) + 1)))
    return loss

In [35]:
def threshold_binary_accuracy(y_true, y_pred, threshold = 0.1):
  threshold = math_ops.cast(threshold, y_pred.dtype)
  y_pred = math_ops.cast(y_pred > threshold, y_pred.dtype)
  return K.mean(math_ops.equal(y_true, y_pred), axis=-1)

In [36]:
def ber(y_true, y_pred):
    return(K.mean(K.cast(K.not_equal(y_true, K.round(y_pred)),dtype='float32')))


input_b = Input(shape = (2,))
#dense1=Dense(4,activation='linear', use_bias = False)(input_b)
msg_out=Dense(2,activation='linear', use_bias = False)(input_b)

model = Model(inputs=input_b,outputs=msg_out)
model.compile(optimizer='adam',loss = RegularizationLoss(0.3),metrics = [threshold_binary_accuracy])
model.summary()

model.fit(y_train,x_train,batch_size = 1,nb_epoch=10000,verbose=2,shuffle=True,validation_data=(y_validate, x_validate))


ValueError: Tried to convert 'input' to a tensor and failed. Error: Argument must be a dense tensor: [array([[ 0.7194662 ,  0.839327  ],
       [-0.36686307, -0.10236776]], dtype=float32)] - got shape [1, 2, 2], but wanted [1].

In [49]:
model.get_weights()

[array([[-6.999995 ,  3.       ],
        [ 4.9999995, -2.       ]], dtype=float32)]

(2, 2)

array([[ 9.9999946e-01, -8.9674722e-07],
       [ 6.0444039e-01,  1.1326264e+00]], dtype=float32)